In [2]:
import pandas as pd
import numpy as np


In [3]:
path = "../00_Source/US_VitalStatistics/"
filename = "Underlying Cause of Death, "

#  first file
main_dataframe = pd.read_csv(path + filename + "2003.txt", sep="\t")

for year in range(2004, 2016):
    """concatenate the rest of the files, drop notes, doesnt seem to have any info,
    get rid of code year as well,same as year"""
    main_dataframe = pd.concat(
        [main_dataframe, pd.read_csv(path + filename + str(year) + ".txt", sep="\t")]
    )

main_dataframe.drop(["Notes", "Year Code"], axis=1, inplace=True)

# rename county to count_and_state to avoid confusion with county in the state fips
main_dataframe.rename(
    columns={"County": "county_and_State", "County Code": "county_fips"}, inplace=True
)
main_dataframe.shape

(57436, 6)

#### Basic Cleaning

In [4]:
"""
- subset to include only drug overdose deaths, get rid of nan
- changing death column to from object to float and only keep rows with death count, year to int
 """
main_dataframe = main_dataframe[
    main_dataframe["Drug/Alcohol Induced Cause Code"].str[0].isin(["D"])
]
main_dataframe = main_dataframe.replace("Missing", np.nan).astype(
    {"Deaths": float, "Year": int, "county_fips": int}
)

main_dataframe = main_dataframe[main_dataframe["Deaths"].notnull()]

main_dataframe.shape


(10407, 6)

In [7]:
# collapse the datatset to get the total death per year for each county_state_fips
collapsed = main_dataframe.groupby(
    ["county_and_State", "county_fips", "Year"], as_index=False
)['Deaths'].apply(lambda x: x.sum())

# check for autauga county, alabama
collapsed[collapsed["county_fips"] == 1003]



,county_and_State,county_fips,Year,Deaths
267,"Baldwin County, AL",1003,2003,10.0
268,"Baldwin County, AL",1003,2004,18.0
269,"Baldwin County, AL",1003,2005,14.0
270,"Baldwin County, AL",1003,2006,11.0
271,"Baldwin County, AL",1003,2007,24.0
272,"Baldwin County, AL",1003,2008,19.0
273,"Baldwin County, AL",1003,2009,29.0
274,"Baldwin County, AL",1003,2010,24.0
275,"Baldwin County, AL",1003,2011,23.0
276,"Baldwin County, AL",1003,2012,24.0


In [6]:
# read in the state fips codes
state_fips = pd.read_csv("../00_Source/state_and_county_fips.csv")

# drop NaN values from state
state_fips = state_fips[state_fips["state"].notnull()]

# column 'name' to county
state_fips.rename(columns={"name": "county"}, inplace=True)
